In [4]:
import cv2 , time
import numpy as np
from skimage import io
from skimage.measure import find_contours
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from skimage.draw import line

In [71]:
cap = cv2.VideoCapture(0)
lower = np.array([0, 10, 60], dtype = "uint8") 
upper = np.array([20, 150, 255], dtype = "uint8")
#face_cascade = cv2.CascadeClassifier('face.xml')
#subtractor = cv2.createBackgroundSubtractorMOG2(history=120, varThreshold=20, detectShadows=True)
count =1
#maskprev = None
# initialize weight for running average
#aWeight = 0.5
SE = np.ones((15,15))
SEH = np.ones((4,2))

'''
_,bg = cap.read()
bg = cv2.flip(bg,1)
frame = cv2.medianBlur(bg,5)
blur = cv2.GaussianBlur(bg,(11,11),3)
bg = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)        
'''        

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame = cv2.medianBlur(frame,3)
    blur = cv2.GaussianBlur(frame,(3,3),3)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    #hsv[:,:,2]= cv2.equalizeHist(hsv[:,:,2])
    #hsv=cv2.cvtColor(hsv, cv2.COLOR_BGR2HSV)
    #v = hsv[:,:,2]
    #v[v>200] =v[v>200]-70
    #hsv[:,:,2] = v
    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.medianBlur(mask,7)
    #mask=cv2.erode(mask,SE,iterations = 7)
    
    #mask=cv2.dilate(mask,SEH,iterations = 7)
    
    #mask=cv2.erode(mask,SE,iterations = 5)
    
    cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(frame,cnts, -1, (0, 0, 255))
    for contour in cnts:
        area = cv2.contourArea(contour)
        #print(contour)
        if area >10000 and area <25000:
            x,y,w,h = cv2.boundingRect(contour)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            Xvalues = contour[:,0,0]
            Yvalues = contour[:,0,1]      
            Xmin = int(np.min(Xvalues))
            Xmax = int(np.max(Xvalues))
            Ymin = int(np.min(Yvalues))
            Ymax = int(np.max(Yvalues))
            
            #print(np.count_nonzero(mask[x:x+w,y:y+h] == 255))
            if np.count_nonzero(mask[x:x+w,y:y+h] == 255) >500 and np.count_nonzero(mask[x:x+w,y:y+h] == 255) <15000: 
                #print(x,y)
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),5) 
                gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
                if w >100:
                    #cv2.rectangle(frame,(Xmin,Ymin),(Xmax,Ymax),(0,0,255),10) 
                    segmented = np.zeros((Ymax-Ymin,Xmax-Xmin))
                    segmented2 = np.zeros((Ymax-Ymin,Xmax-Xmin))
                    segmented = gray[Xmin:Xmax,Ymin:Ymax]
                    segmented2 = mask[Xmin:Xmax,Ymin:Ymax]
                    #segmented2=cv2.morphologyEx(segmented2, cv2.MORPH_OPEN, SE)
                    #segmented2=cv2.morphologyEx(segmented2, cv2.MORPH_CLOSE, SE)
                    
                    #cv2.floodFill(segmented2,np.zeros((segmented2.shape[0]+2,segmented2.shape[1]+2),np.uint8),(0,0),255)
                if segmented.shape[0]>0 and segmented.shape[1]>0 and segmented2.shape[0]>0 and segmented2.shape[1]>0:
                    detection(mask,contour)
                    #kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
                    #segmented2 = cv2.morphologyEx(segmented2,cv2.MORPH_OPEN,kernel)
                    #segmented2=cv2.morphologyEx(segmented2, cv2.MORPH_CLOSE, SEH)
                    #segmented2=cv2.morphologyEx(segmented2, cv2.MORPH_CLOSE, SEH)
                    
                    #cv2.imshow("segmentedMask",segmented2 )
                    #segmented=Median(segmented,(3,3))
                    #segmented=cv2.dilate(segmented,SE,iterations = 7)
                    #segmented=cv2.erode(segmented,SE,iterations = 7)
                    #segmented=cv2.dilate(segmented,SE,iterations = 3)
                    
                    #thresh = threshold_otsu(segmented) #Calculate The Best Threshold gray image from 0 - 255
                    #ret3,th3 = cv2.threshold(segmented,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                    #segmented = segmented>thresh
                    #th3=1-th3
                    #dsize = (429, 429)
                    # resize image
                    #segmented = cv2.resize(segmented, dsize)
                    #segmented=cv2.bitwise_and(segmented,bg[x:x+w,y:y+h])
                    #segmented=cv2.absdiff(segmented,bg[x-50:x+w+50,y-50:y+h+50])
                    ret3,segmented = cv2.threshold(segmented,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                    
                    segmented=cv2.erode(segmented,SE,iterations = 15)
                    segmented=cv2.morphologyEx(segmented, cv2.MORPH_CLOSE, SEH)
                    #segmented=cv2.dilate(segmented,SEH,iterations = 11)
                    #cv2.imshow("segmented",segmented )
        else:
            x,y,w,h = cv2.boundingRect(contour)
            cv2.rectangle(mask,(x,y),(x+w,y+h),(0,0,0),-1)
    #mask2 = subtractor.apply(hsv)
    #mask_inv = cv2.bitwise_not(mask)
    '''
    if num_frames < 30:
        run_avg(mask, aWeight)
        print(bg.shape)
        cv2.imshow("bg",bg)
    '''
    #else:
        # segment the hand region
        #frame = cv2.absdiff()
   
    #mask[x:x+w,y:y+h]=0
    #mask2 = subtractor.apply(mask)
    #maskprev=mask
    
    #result = cv2.bitwise_and(frame, frame, mask=mask)
    
    cv2.imshow("frame", frame)
    cv2.imshow("mask", mask)

    #cv2.imshow("result", result)
    
    # observe the keypress by the user
    keypress = cv2.waitKey(1) & 0xFF

    # if the user pressed "q", then stop looping
    if keypress == ord("q"):
        break

# free up memory
cap.release()
cv2.destroyAllWindows()


In [70]:
def convex_hull(points):
    """Computes the convex hull of a set of 2D points.

    Input: an iterable sequence of (x, y) pairs representing the points.
    Output: a list of vertices of the convex hull in counter-clockwise order,
    starting from the vertex with the lexicographically smallest coordinates.
    Implements Andrew's monotone chain algorithm. O(n log n) complexity.
    """

    # Sort the points lexicographically (tuples are compared lexicographically).
    # Remove duplicates to detect the case we have just one unique point.
    points = sorted(set(points),key=lambda x: (x[1], x[0]))
    # Boring case: no points or a single point, possibly repeated multiple times.
    if len(points) <= 1:
        return points

    # 2D cross product of OA and OB vectors, i.e. z-component of their 3D cross product.
    # Returns a positive value, if OAB makes a counter-clockwise turn,
    # negative for clockwise turn, and zero if the points are collinear.
    def cross(o, a, b):
        return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])

    indices_of_hulls_lower = []
    # Build lower hull 
    lower = []
    for i in range(len(points)):
        while len(lower) >= 2 and cross(lower[-2], lower[-1], points[i]) <= 0:
            lower.pop()
            indices_of_hulls_lower.pop()
        lower.append(points[i])
        indices_of_hulls_lower.append(i)

    indices_of_hulls_upper = []
    # Build upper hull
    upper = []
    for i in range(len(points)-1,0,-1):
        while len(upper) >= 2 and cross(upper[-2], upper[-1], points[i]) <= 0:
            upper.pop()
            indices_of_hulls_upper.pop()
        upper.append(points[i])
        indices_of_hulls_upper.append(i)

    # Concatenation of the lower and upper hulls gives the convex hull.
    # Last point of each list is omitted because it is repeated at the beginning of the other list. 
    return lower[:-1] , upper[:-1] , indices_of_hulls_lower[:-1],indices_of_hulls_upper[:-1], points


In [6]:
# Function to get defects
def convexity_defects(hulls,indices_of_hulls,contour):
    # contains the defects
    defects=[]
    for i in range(len(hulls)-1):
            maximum=-1
            maximum_point=(-1,-1)
            mid_point=((hulls[i][0]+hulls[i+1][0])//2 -150,(hulls[i][1]+hulls[i+1][1])//2)
            point=(-1,-1)
            for j in range (indices_of_hulls[i],indices_of_hulls[i+1],1):
                if point[1] != contour[j][1]:
                    point=contour[j]
                    distance=np.sqrt( pow(mid_point[0]-point[0],2) + pow(mid_point[1]-point[1],2))
                    # Make the defect the farthest point from contours
                    if distance > maximum:
                        maximum=distance
                        maximum_point=point
            # Adding defects to the array of defects
            dist_first_pnt = np.sqrt( pow(hulls[i][0]-maximum_point[0],2) + pow(hulls[i][1]-maximum_point[1],2))
            dist_second_pnt = np.sqrt( pow(hulls[i+1][0]-maximum_point[0],2) + pow(hulls[i+1][1]-maximum_point[1],2))
            if maximum > -1 and (dist_first_pnt > 20 and dist_second_pnt > 20):
                defects.append(maximum_point)
    return defects

In [37]:
def detection(card,contours):  
    '''
    card=io.imread('hand\\Mahmoud\\seven.jpg')
    #card=cv2.flip(card,1)
    card = cv2.cvtColor(card, cv2.COLOR_BGR2GRAY)
    # Resize image
    scale_percent = 60 # Percent of original size
    width = int(card.shape[1] * scale_percent / 100)
    height = int(card.shape[0] * scale_percent / 100)
    dim = (429, 429)
    card = cv2.resize(card, dim, interpolation = cv2.INTER_AREA)
    # Convert image to binary
    ret, threshold = cv2.threshold(card, 200, 255, cv2.THRESH_BINARY)
    # Find all contours of the shape
    '''
    #contours, hiearchy = cv2.findContours(card,mode=2,method=1)
    tempcontours=contours
    #print(len(contours))
    # Contour area
    areacnt = cv2.contourArea(contours[0])
    # Make contours tuples 
    cnt1 = contours[:,0,0] 
    cnt2 = contours[:,0,1] 
    cnt = [(cnt2[i], cnt1[i]) for i in range(0, len(cnt2))]

    # Return contours arranged from left to right
    lower_hull , upper_hull , indices_of_lower_hulls , indices_of_upper_hulls , contours = convex_hull ( cnt )

    # Append lowerhull + upperhull
    hull = lower_hull + upper_hull

    # To reshape hulls to be able to get its area
    arrhull=np.array([0,0])
    result=np.array([0,0])
    l=0
    for i in hull:
        row_to_be_added = np.array([i[0], i[1]]) 
        # Adding row to numpy array 
        if l != 0:
            result = np.vstack ((arrhull, row_to_be_added) ) 
            arrhull=result
        else:
            arrhull=row_to_be_added
        l+=1

    # Area of the hull
    areahull = cv2.contourArea(arrhull)

    # Ratio between contour area and hull area
    ratio = areacnt/areahull

    #draw lines between hulls
    for i in range (len(hull)-1):
        rr, cc = line(int(hull[i][0]),int(hull[i][1]), int(hull[i+1][0]),int(hull[i+1][1]))
        card[rr,cc]=255

    stacked_img = np.stack((card,)*3, axis=-1)

    #New hulls including the last point
    new_hulls=[]
    new_ind=[]
    # Reverse to start from left to right
    upper_hull.reverse()
    indices_of_upper_hulls.reverse()
    #making hulls to be sent to convexity defects to calculate defects
    for i in range(len(upper_hull)-1):
        if np.sqrt(pow(upper_hull[i][0]-upper_hull[i+1][0],2)+pow(upper_hull[i][1]-upper_hull[i+1][1],2)) > 50:
            if upper_hull[i][0]-upper_hull[i+1][0] < 150:
                new_hulls.append(upper_hull[i])
                new_ind.append(indices_of_upper_hulls[i])

    #if len(new_hulls)>0 and upper_hull[-1][0]-new_hulls[-1][0] < 130:       
    new_hulls.append(upper_hull[-1])
    new_ind.append(indices_of_upper_hulls[-1])

    #Defects to be draw in the img
    defects=convexity_defects(new_hulls,new_ind,contours)
    font = cv2.FONT_HERSHEY_SIMPLEX

    number=GetNumber(len(tempcontours),len(new_hulls),len(defects),ratio)
    cv2.putText(frame,number,(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

'''
    for i in defects:
        stacked_img[int(i[0]):int(i[0]+15),int(i[1]):int(i[1]+15)]=(0,0,255)

    for i in new_hulls:
        stacked_img[int(i[0]):int(i[0]+10),int(i[1]):int(i[1]+10)]=(255,0,0)
    plt.figure(figsize=(8,8))

    io.imshow(stacked_img)
    io.show()

    print(GetNumber(len(tempcontours),len(new_hulls),len(defects),ratio))

    print(len(defects),len(new_hulls))
'''

'\n    for i in defects:\n        stacked_img[int(i[0]):int(i[0]+15),int(i[1]):int(i[1]+15)]=(0,0,255)\n\n    for i in new_hulls:\n        stacked_img[int(i[0]):int(i[0]+10),int(i[1]):int(i[1]+10)]=(255,0,0)\n    plt.figure(figsize=(8,8))\n\n    io.imshow(stacked_img)\n    io.show()\n\n    print(GetNumber(len(tempcontours),len(new_hulls),len(defects),ratio))\n\n    print(len(defects),len(new_hulls))\n'

In [29]:
def GetNumber(contourscnt,hulls,defects,ratio):
    if contourscnt ==2:
        return'Zero'
    elif defects ==0:
        if hulls==0:
            return 'Transition'
        elif hulls==2 and ratio > 0.75 and ratio < 0.85:
            return 'One'
    elif defects==1:
        if ratio<0.68:
            return 'Seven'
        else:
            return 'Two'
    elif defects == 2:
        return 'Three'
    elif defects == 3:
        return 'Four'
    elif defects == 4:
        return 'Five'
